In [6]:
import os
import anthropic

client = anthropic.Anthropic(
    api_key=os.getenv('ANTHROPIC_API_KEY'),
)

model_name = "claude-3-5-sonnet-20240620"
batch_size = 100


def batch_translate(bigrams):
    message = client.messages.create(
        # model="claude-3-opus-20240229",
        model=model_name,
        max_tokens=4000,
        temperature=0,
        system="Выступи в роли знатока кабардинского языка и помоги мне перевести строки на русский язык. \n",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"""Результат ожидаю без лишних объяснений, строго перевод на русском языке. \n\nСтроки для перевода будут разделены знаком \\n (переноса строки), в ответе перевод должен быть так же разделен \\n.  \n\nКаждая новая строка, это отдельное по смыслу предложение - соединять их и тем более пытатся их переводить вместе не стоит. \n\nНа выходе переведенных строк должно быть строго как и входных. \n\nЕсли не можешь перевести строку, ставь пустую строку \"\" чтобы остальные переведы остались под тем же индексом, это супер важно. \n\nНе пиши в ответе вводную информацию \"Перевод на русский язык:\" - только результат перевода. \n\n\nпокажу пример данных чтобы ты понимал формат. \n- на вход:\nнэхърэ нэхъ\nсыт хуэдэ\n\n- на выходе хочу получить:\nчем больше\nкакой, какая, какое\n\nследующим сообщением пришлю входные данные которые нужно будет тебе перевести. Буду присылать батчами по {batch_size} строк. Ожидаю получить ровно {batch_size} строк. Это важно для моего скрипта обработки данных: batches = df.groupby(df.index // 100)\n\nfor i, (name, group) in tqdm(enumerate(batches), total=len(batches)):\n    group_result = batch_translate(group['word'].tolist())\n    try:\n        group_result_df = pd.DataFrame({{\n            'source': group['word'].tolist(),\n            'translated': group_result\n        }})\n        group_result_df.to_csv(f'../data/processed/word_freqs/freq_1000000_oshhamaho_{i}_{model_name}.csv', index=False)\n    except Exception:\n        continue"""
                    }
                ]
            },
            {
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        "text": "Понял, жду строки для перевода. "
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "\n".join(bigrams)
                    }
                ]
            }
        ]
    )

    result = message.content[0].text.split('\n')
    return result

In [8]:
from tqdm import tqdm
import pandas as pd

df = pd.read_csv('../data/processed/word_bigrams/oshhamaho.csv')
df.sort_values('w_bigram', inplace=True)
df

,w_bigram,freq
341,100 ирокъу,14
2850,105-рэ ирокъу,3
2527,150-рэ щрикъум,3
1091,1557 гъэм,5
2049,1560 гъэм,3
...,...,...
49,ящыщ зыщ,52
587,ящыщ куэд,9
1227,ящыщ куэдым,5
1653,ящыщу нэхъ,4


In [3]:
def process_batch(batch_data):
    name, group = batch_data
    group_result = batch_translate(group['w_bigram'].tolist())
    try:
        group_result_df = pd.DataFrame({
            'w_bigram': group['w_bigram'].tolist(),
            'translated': group_result
        })
        group_result_df.to_csv(f'../data/processed/word_bigrams/oshhamaho_{name}_{model_name}.csv', index=False)
        return True
    except Exception as e:
        print(f"Error processing batch {name}: {str(e)}")
        return False


In [5]:
from concurrent.futures import ThreadPoolExecutor, as_completed

batches = df.groupby(df.index // batch_size)

with ThreadPoolExecutor(max_workers=8) as executor:
    future_to_batch = {executor.submit(process_batch, (name, group)): name
                       for name, group in batches
                       if name >= 0
                       }

    for future in tqdm(as_completed(future_to_batch), total=len(future_to_batch)):
        batch_name = future_to_batch[future]
        try:
            success = future.result()
            if not success:
                print(f"Failed to process batch {batch_name}")
        except Exception as e:
            print(f"Batch {batch_name} generated an exception: {str(e)}")

100%|██████████| 30/30 [00:00<00:00, 118149.41it/s]

Batch 17 generated an exception: name 'i' is not defined
Batch 15 generated an exception: name 'i' is not defined
Batch 0 generated an exception: name 'i' is not defined
Batch 9 generated an exception: name 'i' is not defined
Batch 3 generated an exception: name 'i' is not defined
Batch 27 generated an exception: name 'i' is not defined
Batch 7 generated an exception: name 'i' is not defined
Batch 13 generated an exception: name 'i' is not defined
Batch 29 generated an exception: name 'i' is not defined
Batch 25 generated an exception: name 'i' is not defined
Batch 11 generated an exception: name 'i' is not defined
Batch 23 generated an exception: name 'i' is not defined
Batch 21 generated an exception: name 'i' is not defined
Batch 20 generated an exception: name 'i' is not defined
Batch 18 generated an exception: name 'i' is not defined
Batch 16 generated an exception: name 'i' is not defined
Batch 5 generated an exception: name 'i' is not defined
Batch 8 generated an exception: name